## Chapter2: 시계열 데이터 수집 및 전처리

이번 장에서는 시계열 데이터 수집, 전처리하는 방법을 배운다.

### 목차
- 온라인 저장소에서 시계열 데이터 찾기
- 시계열을 고려하지 않고 수집된 데이터에서 시계열 데이터 수집 및 전처리
- timestamp 다루는 방법


### 2.1. 시계열 데이터 찾기

시계열 데이터는 목적에 따라 크게

1. 학습, 실험 목적으로 생성된 데이터
    - [UCI 머신러닝 저장소 - 시계열](https://archive.ics.uci.edu/ml/datasets.php?format=&task=&att=&area=&numAtt=&numIns=&type=ts&sort=nameUp&view=table): 결근, 공기질 등.
    - [UEA, UCR 시계열 분류 저장소](https://perma.cc/56Q5-YPNT): [요가 동작 분류 작업](https://perma.cc/U6MU-2SCZ), [와인 데이터셋](https://perma.cc/Y34-UGMD) 등
    - 정부 시계열 데이터셋: 미국 국립환경정보센터NCEI, 미국 노동통계국, 미국 질병통제에방센터CDC, 세인트루이스 연방준비은행, 한국 기상청 등. 경제와 기후, 범죄에 대한 정부 지출과 각 범죄율을 병렬로 다변량 시계열을 사용하면서 분석할 수 있다. 하지만 여러 요인이 작용하므로 학습용으로는 적합하지 않을 수 있음.
    - [CompEngine](https://www.comp-engine.org/#!browse/category): 시계열 데이터를 스스로 조직화하는 데이터베이스.
    - R의 Mcomp, M4comp2018 패키지, [CRAN 저장소](https://perma.cc/2694-D79K)의 시계열 데이터 섹션
2. 시계열이 아닌 데이터에서 시계열 데이터 생성: timestamp를 시계열로 변환 및 결합
    - timestamp가 찍힌 기록: 임의의 시점과 그 시점 이후의 시점 타임 스탬프(접근 등)으로 시간의 변화량을 모델링할 수 있음
    - 시간 없는 측정으로 시간 대체: 데이터가 시간을 명시적으로 포함하지 않아도, 데이터의 숨은 논리로 유추할 수 있는 경우. 센서 주기 등
    - 물리적 흔적: 디지털로 저장된 흔적을 이미지나 데이터베이스에 저장됨

으로 나눌 수 있다.


- 유의: 데이터에 구체적 '시간'이 없더라도, **일련의 정렬된 형태를 띈다면 시계열 적용 가능**






### 2.2. 흩어진 데이터를 시계열 데이터로 만들기

NGO에서 시계열 분석을 한다고 할 때, SQL로도 충분히 시계열에 대한 현황 분석을 할 수 있다.
- 메일을 읽었는지:  빈도, 응답 시간에 따라 메일 수신의 피로감 파악
- 기부 멤버십 중단 경험: 시계열 기부 예측 
- 거래 내역: 어떤 특정한 시점에 구매할지 예측

하지만 보통 데이터베이스 스키마를 설계할 때, 시계열 분석을 고려하지 않아 따로 시계열을 수집하고 구성해야한다.

아래 데이터의 시간 구분은 다음과 같다.
- YearJoined: 연간 회원 상태
- emails: 이메일 열람 관련 주간 누적 기록
- donations: 기부가 이루어진 순간의 타임스탬프

#### 정리

1. **해결하고자 하는 문제에 맞는 형태로 데이터 간격 교정**
2. **사전 관찰을 피하기 위해 timestamp를 사용하지 않는 방법 모색**
3. **아무 일이 없더라도 관련된 모든 기간을 기록**
4. **사전관찰을 피하기 위해 아직 알아서는 안되는 timestamp를 다시 한번 배제**

In [3]:
import pandas as pd
import io
import requests

In [4]:
YearJoined = pd.read_csv("https://raw.githubusercontent.com/PracticalTimeSeriesAnalysis/BookRepo/master/Ch02/data/year_joined.csv")
YearJoined.head()

,user,userStats,yearJoined
0,0,silver,2014
1,1,silver,2015
2,2,silver,2016
3,3,bronze,2018
4,4,silver,2018


In [5]:
emails = pd.read_csv("https://raw.githubusercontent.com/PracticalTimeSeriesAnalysis/BookRepo/master/Ch02/data/emails.csv")
emails.head()

,emailsOpened,user,week
0,3.0,1.0,2015-06-29 00:00:00
1,2.0,1.0,2015-07-13 00:00:00
2,2.0,1.0,2015-07-20 00:00:00
3,3.0,1.0,2015-07-27 00:00:00
4,1.0,1.0,2015-08-03 00:00:00


In [6]:
donations = pd.read_csv("https://raw.githubusercontent.com/PracticalTimeSeriesAnalysis/BookRepo/master/Ch02/data/donations.csv")
donations.head()

,amount,timestamp,user
0,25.0,2017-11-12 11:13:44,0.0
1,50.0,2015-08-25 19:01:45,0.0
2,25.0,2015-03-26 12:03:47,0.0
3,50.0,2016-07-06 12:24:55,0.0
4,50.0,2016-05-11 18:13:04,1.0


1000명의 모든 회원이 하나의 상태만 가진다. 따라서 year Joined는 가입연도와 현재 혹은 가입 당시의 상태를 나타낼 가능성이 높음. 정확하게 분석하려면 데이터 파이프라인을 알아야한다.

 - **사전관찰**: 과거 데이터 분석에 현재 회원의 상태를 적용하면 알 수 없는 시계열 모델에 무언가를 입력하는 꼴이 된다.
    - 시계열에서 사전관찰: 미래의 어떤 사실을 안다는 의미로 사용. 데이터를 통해 실제로 알아야하는 시점보다 더 일찍 미래에 대한 사실을 발견. 미래에 일어난 일에 대한 정보가 과거 모델 초기 동작에 영향을 주는 방법. 자동화된 코드나 통게 테스트가 없어서 스스로 고민해야함.


In [7]:
# 28 page
YearJoined.groupby('user').count().groupby('userStats').count()

,yearJoined
userStats,
1,1000


- 고려해야할 점
    - 어떤 방식으로 시간을 표현했는지: 주wwek를 나눌 때 1.1로 나누는게 아니라 첫 월요일을 기준으로 하는 등.
    - null 값이 있는지: **시계열에서는 null이 존재해야한다**. 아무일도 발생하지 않는 주 또한 데이터의 일부.



In [8]:
emails[emails.emailsOpened < 1]

,emailsOpened,user,week


- 두 가지 가능성
    1. null이 발생하지 않았거나, 
    2. 모든 회원이 이메일을 열람했다는 이벤트가 적어도 하나 존재

하지만 모든 회원이 매주 이메일 열람은 가능성이 낮다. 따라서 특정 한 회원 데이터를 살펴보면서 판단 가능

In [9]:
emails[emails.user == 998]

,emailsOpened,user,week
25464,1.0,998.0,2017-12-04 00:00:00
25465,3.0,998.0,2017-12-11 00:00:00
25466,3.0,998.0,2017-12-18 00:00:00
25467,3.0,998.0,2018-01-01 00:00:00
25468,3.0,998.0,2018-01-08 00:00:00
25469,2.0,998.0,2018-01-15 00:00:00
25470,3.0,998.0,2018-01-22 00:00:00
25471,2.0,998.0,2018-01-29 00:00:00
25472,3.0,998.0,2018-02-05 00:00:00
25473,3.0,998.0,2018-02-12 00:00:00


일부 주가 누락됨. 17년 12.18. 이후 이메일 열람 기록이 없음.

#### 수학적으로 주차 계산하기

In [10]:
emails.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25488 entries, 0 to 25487
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   emailsOpened  25488 non-null  float64
 1   user          25488 non-null  float64
 2   week          25488 non-null  object 
dtypes: float64(2), object(1)
memory usage: 597.5+ KB


In [11]:
import datetime
date_time_str_max = max(emails[emails.user == 998].week)
date_time_str_min = min(emails[emails.user == 998].week)

date_time_obj_max = datetime.datetime.strptime(date_time_str_max, '%Y-%m-%d %H:%M:%S')
date_time_obj_min = datetime.datetime.strptime(date_time_str_min, '%Y-%m-%d %H:%M:%S')

(date_time_obj_max - date_time_obj_min).days/7

25.0

In [12]:
emails[emails.user == 998].shape

(24, 3)

전체 26주 중 24개 데이터만 있는 것으로 보아 2개 누락. 물론 이는 한 명의 데이터이므로 다른 사람은 다를 수도 있다. null 값은 없지만, 아예 데이터가 수집되지 않은 주는 있다.

- 왜 25가 아니고 26주?
    - 마지막에 1을 더할지 말지 고민해야함
    - 4월 7, 14, 21, 28 -> 4일
    - (마지막 날 - 처음 날) / 7 = (28-7) / 7 = 3
    - 처음, 혹은 마지막 날을 빼먹음. 따라서 1 더해주어야 함.


#### 판다스 multi index를 이용해 주차 구하기

In [13]:
complete_idx = pd.MultiIndex.from_product((set(emails.week), set(emails.user)))

In [14]:
complete_idx

MultiIndex([('2018-03-19 00:00:00',   1.0),
            ('2018-03-19 00:00:00',   3.0),
            ('2018-03-19 00:00:00',   5.0),
            ('2018-03-19 00:00:00',   6.0),
            ('2018-03-19 00:00:00',   9.0),
            ('2018-03-19 00:00:00',  10.0),
            ('2018-03-19 00:00:00',  14.0),
            ('2018-03-19 00:00:00',  16.0),
            ('2018-03-19 00:00:00',  20.0),
            ('2018-03-19 00:00:00',  21.0),
            ...
            ('2016-10-03 00:00:00', 973.0),
            ('2016-10-03 00:00:00', 977.0),
            ('2016-10-03 00:00:00', 982.0),
            ('2016-10-03 00:00:00', 984.0),
            ('2016-10-03 00:00:00', 987.0),
            ('2016-10-03 00:00:00', 991.0),
            ('2016-10-03 00:00:00', 992.0),
            ('2016-10-03 00:00:00', 993.0),
            ('2016-10-03 00:00:00', 995.0),
            ('2016-10-03 00:00:00', 998.0)],
           length=93247)

In [15]:
all_email = emails.set_index(['week', 'user']).reindex(complete_idx, fill_value=0).reset_index()
all_email.columns = ['week', 'user', 'emailsOpened']

In [16]:
all_email[all_email.user ==998].sort_values('week')

,week,user,emailsOpened
48509,2015-02-09 00:00:00,998.0,0.0
22098,2015-02-16 00:00:00,998.0,0.0
28566,2015-02-23 00:00:00,998.0,0.0
45814,2015-03-02 00:00:00,998.0,0.0
33956,2015-03-09 00:00:00,998.0,0.0
...,...,...,...
18325,2018-04-30 00:00:00,998.0,3.0
66296,2018-05-07 00:00:00,998.0,3.0
59828,2018-05-14 00:00:00,998.0,3.0
58750,2018-05-21 00:00:00,998.0,3.0


998번 회원의 경우 가입 직후에 email을 열지 않았다. 0이 아닌 숫자가 나오기 이전 데이터를 null로 처리하는 방법은 다음과 같다.

먼저 회원별로 처음 메일을 열기 시작한 시점을 알아보자

In [17]:
# cutoff dates
cutoff_dates = emails.groupby('user').week.agg(['min','max']).reset_index()

cutoff_dates = cutoff_dates.reset_index()

cutoff_dates

,index,user,min,max
0,0,1.0,2015-06-29 00:00:00,2018-05-28 00:00:00
1,1,3.0,2018-03-05 00:00:00,2018-04-23 00:00:00
2,2,5.0,2017-06-05 00:00:00,2018-05-28 00:00:00
3,3,6.0,2016-12-05 00:00:00,2018-05-28 00:00:00
4,4,9.0,2016-07-18 00:00:00,2018-05-28 00:00:00
...,...,...,...,...
534,534,991.0,2016-10-24 00:00:00,2016-10-24 00:00:00
535,535,992.0,2015-02-09 00:00:00,2015-07-06 00:00:00
536,536,993.0,2017-09-11 00:00:00,2018-05-28 00:00:00
537,537,995.0,2016-09-05 00:00:00,2018-05-28 00:00:00


우리가 봤던 998번 회원은 2015년 2월 9일에 가입했지만 2017년 12월 4일부터 메일을 열기 시작했다!

이제 처음 메일을 열기 전 데이터를 삭제하자

In [18]:
import warnings
warnings.filterwarnings('ignore')

for _, row in cutoff_dates.iterrows():
  user        = row['user']
  start_date  = row['min']
  end_date    = row['max']
  all_email.drop(all_email[all_email.user == user][all_email.week < start_date].index, inplace=True)
  all_email.drop(all_email[all_email.user == user][all_email.week > end_date].index, inplace=True)

이번에는 기부금을 주별로 취합해 이메일 반응과 상관관계를 알아봅시다.

1. donation은 순간의 timestamp로 되어있으므로 주별로 취합(down sampling)
2. 이번 주의 donation에 대한 예측 변수: donation 이전 주의 열람 email 개수
3. 한 주에 여러번 기부하는 사람은 흔치 않으므로 기부 건수 = 기부자 수로 간주

In [19]:
# 33
donations.timestamp = pd.to_datetime(donations.timestamp)   # 문자열 -> 시계열

donations.set_index('timestamp', inplace=True) 

# '주'를 월요일로 고정한 후 재배정 -> 이메일 주간 날짜 데이터와 일치 시키기
agg_don = donations.groupby('user').apply(lambda df: df.amount.resample("W-MON").sum().dropna())

agg_don

user   timestamp 
0.0    2015-03-30      25.0
       2015-04-06       0.0
       2015-04-13       0.0
       2015-04-20       0.0
       2015-04-27       0.0
                      ...  
995.0  2017-09-11       0.0
       2017-09-18       0.0
       2017-09-25       0.0
       2017-10-02    1000.0
998.0  2018-01-08      50.0
Name: amount, Length: 32352, dtype: float64

In [27]:
merged_df = pd.DataFrame()

for user, user_email in all_email.groupby('user'):
    ## 특정 회원의 기부 데이터 추출
    user_donations = agg_don[agg_don.index.get_level_values('user') == user]
  
    ## 기부 데이터 인덱스를 timestamp로 설정
    user_donations = user_donations.droplevel(0)
    # user_donations.set_index('timestamp', inplace = True)  
  
    user_email = all_email[all_email.user == user]
    user_email.sort_values('week', inplace=True)
    user_email.set_index('week', inplace=True)

    df = pd.merge(user_email, user_donations, how='left', left_index=True, right_index=True)
    df.fillna(0)

    merged_df = merged_df.append(df.reset_index()[['user', 'week', 'emailsOpened', 'amount']])

merged_df.head()

,user,week,emailsOpened,amount
0,1.0,2015-06-29 00:00:00,3.0,NaN
1,1.0,2015-07-06 00:00:00,0.0,NaN
2,1.0,2015-07-13 00:00:00,2.0,NaN
3,1.0,2015-07-20 00:00:00,2.0,NaN
4,1.0,2015-07-27 00:00:00,3.0,NaN


In [28]:
# 998번 회원만 고르기

df = merged_df[merged_df.user == 998]

## .shift(n): amount 열의 기록을 뒤로 n만큼 미룬다.
df['target'] = df.amount.shift(1)
df = df.fillna(0)
df.head(7)

,user,week,emailsOpened,amount,target
0,998.0,2017-12-04 00:00:00,1.0,0.0,0.0
1,998.0,2017-12-11 00:00:00,3.0,0.0,0.0
2,998.0,2017-12-18 00:00:00,3.0,0.0,0.0
3,998.0,2017-12-25 00:00:00,0.0,0.0,0.0
4,998.0,2018-01-01 00:00:00,3.0,0.0,0.0
5,998.0,2018-01-08 00:00:00,3.0,0.0,0.0
6,998.0,2018-01-15 00:00:00,2.0,0.0,0.0


### 2.3 Timestamp의 문제

#### 2.3.1 timestamp 자체는 유용하지만 다루기 매우 어렵다.

- 무엇에 관한 timestamp인지 알기 어렵다.
    - 생성 과정
    - 생성 방법
    - 생성 시기

어느날 식단 기록 어플리케이션에 모두 같은 시간에 샐러드, 디톡스 주스, 현미밥과 각종 반찬을 기록했다면 한 끼에 먹은걸까? 하루 종일 먹은걸까? 어느 지역 기준의 시간인지도 모호하다. 이 데이터베이스를 설계하지 않았기 때문에 연구자는 timestamp의 생성 과정, 방법, 시기가 정해지는 매커니즘을 알기 어렵다. 가장 쉬운 방법은 정보 수집 코드를 모두 해석하거나, 코드를 작성한 사람에게 묻는 작업이다.

#### 2.3.2 timestamp를 추측해 데이터 이해하기
몇가지 추론에 도움이 되는 프로세스는 다음과 같다.
1. timestamp에 대한 가설 세우기: 동일하거나, 이례적 패턴 파악
2. 가설 시험: 현지 시간인지 표준 시간인지, 시간이 사용자의 행동을 반영하는지 외부 제약을 반영하는지 등


- **현지시간? 표준시간?**
    - 현지시간: 말 그대로 현지 시간. 하루 위주의 패턴 존재.
    - 협정 세계시간Coordinated Universal Time(UTC): 빈번한 사용 요일, 시간을 파악해 절대적 시간에 기반한 추론. 2시 7시 14시에 식사했다는 데이터라면 예컨대 7시, 12시, 19시에 식사했다고 추측할 수 있음.

- **dt**: 시간 차이를 의미. 각 사용자의 시간대를 추정할 수 있는 특징feature. dt가 길다면 하루 중 밤 시간대를 추측할 수 있음. 


In [ ]:
## dt 구하는 예시
# df['dt'] = df.time - df.time.shift(-1)

# df.head()

### 2.4. 데이터 정리
- 결측치: 
    - 대치법: 전체 관측에 기반해 누락 데이터 채워 넣기
    - 보간법: 대치법의 일종. 인접 데이터를 사용해 누락 데이터 추정
    - 삭제: 누락 데이터를 사용하지 않는 방법
- 시계열 빈도 변경: 업샘플링, 다운 샘플링
- 데이터 평활
- 데이터 계절적 변동 문제 해결
- 의도치 않은 사전관찰 방지

### 2.4.3. 지수 평활Smoothing

In [1]:
import pandas as pd

air = pd.read_csv('C:/Users/jung6/Downloads/AirPassengers.csv', parse_dates = True, header = None)
air.columns = ['Date', 'Passengers']

## thanks to Mark Wilson for this code correction
## ewm: 다양한 감쇠요인 적용
air['Smooth.5'] = air.ewm(alpha=0.5).mean().Passengers
air['Smooth.1'] = air.ewm(alpha=0.1,).mean().Passengers
air['Smooth.9'] = air.ewm(alpha=0.9,).mean().Passengers

In [2]:
air

,Date,Passengers,Smooth.5,Smooth.1,Smooth.9
0,1949-01,112,112.000000,112.000000,112.000000
1,1949-02,118,116.000000,115.157895,117.454545
2,1949-03,132,125.142857,121.372694,130.558559
3,1949-04,129,127.200000,123.590579,129.155716
4,1949-05,121,124.000000,122.957974,121.815498
...,...,...,...,...,...
139,1960-08,606,582.096411,468.874660,606.665454
140,1960-09,508,545.048205,472.787195,517.866545
141,1960-10,461,503.024103,471.608475,466.686655
142,1960-11,390,446.512051,463.447626,397.668665


장기 추세 데이터에서 단순 지수평활은 제대로 예측하기 어렵다. 이를 보완하기 위해 홀트, 홀트-윈터스 평활, 칼만 필터, 국소추정산점도평활LOESS을 이용할 수 있다.

- 홀트 메소드, 홀트-윈터스 평활: 추세를 가진 데이터, 추세와 계절성을 가진 데이터 모두에 적용할 수 있는 지수 평활 방법
- 칼만 필터: 변동성 및 측정 오차의 조합으로 시계열 과정 모델링하여 데이터 평활
- 국소추정산점도평활LOESS: 지역적으로 데이터를 평활하는 비모수적 방법. 복잡해지면서 계산 비용이 증가함. 

칼만 필터와 LOESS는 시간 전후 데이터를 모두 고려하므로 과거로 정보가 유출될 수 있음.

## 2.5 계절성 데이터

- 계절성: 특정 행동 빈도가 안정적으로 반복해서 나타나나는 것. 실제 계절도 포함되지만, 아침, 점심, 저녁 식사 또한 해당한다.

산점도는 평균과 분산 증가를 알 수 있지만, 선 그래프는 계절 변동을 파악하기 쉽다.

- 계절성 데이터: 일련의 동작이 정해진 기간 동안 반복되는 시계열
- 순환성 시계열: 반복적인 동작을 보이긴하지만, 기간이 가변적. 주식시장 호황, 불황은 불확실한 기간이지만 주기가 있음.

-> R에서 계절성seasonal, 추세trend, 나머지remainder가 포함된 그래프 출력

## 2.6 시간대

시간대는 복잡하다.
- 시간대는 정치, 사회적으로 형성
- 언어 간, HTTP 프로토콜을 통해 표준 시간대 정볼르 전송하는 표준 방법이 없음
- 시간대에 이름을 짓거나 일광 절약의 시작과 종료일을 결정하기 위한 단일화도니 프로토콜이 없음
- 일광 절약으로 한 해에 몇 시간씩 시간 중복 발생

파이썬은 자동 시간 검색 함수인 datetime.datetime.now()는 시간대를 인식하지 못함.

In [3]:
import datetime
import pytz

datetime.datetime.utcnow()
# datetime.datetime.utcnow()
# datetime.datetime(2018, 5, 31, 14, 49, 43, 187680)


datetime.datetime(2022, 5, 17, 11, 5, 46, 594072)

In [4]:

datetime.datetime.now()
# >>> >>> datetime.datetime.now()
# datetime.datetime(2018, 5, 31, 10, 49, 59, 984947)
# as we can see, my computer does not return UTC even though there is no time zone attached


datetime.datetime(2022, 5, 17, 20, 5, 46, 756219)

In [5]:

datetime.datetime.now(datetime.timezone.utc)
# >>> datetime.datetime.now(datetime.timezone.utc)
# datetime.datetime(2018, 5, 31, 14, 51, 35, 601355, tzinfo=datetime.timezone.utc)


datetime.datetime(2022, 5, 17, 11, 5, 47, 24462, tzinfo=datetime.timezone.utc)

In [6]:
# timezone 설정
western = pytz.timezone('US/Pacific')
western.zone

'US/Pacific'

API를 통해 시간대를 인지한 시간을 생성하는 두가지 방법
1. localize 이용
2. 한 장소에서 다른 장소의 시간대로 변환하는 방식: pytz


In [7]:
# localize
loc_dt = western.localize(datetime.datetime(2018, 5, 15, 12, 34, 0))
loc_dt

datetime.datetime(2018, 5, 15, 12, 34, tzinfo=<DstTzInfo 'US/Pacific' PDT-1 day, 17:00:00 DST>)

datetime 생성자에 시간대를 직접 전달하는 경우

pytz를 사용하는 경우, 일광 절약이 없는 시간대에서만 원하는 결과를 얻을 수 있다!

In [10]:
# pytz: 일광 절약이 영향을 줌
london_tz = pytz.timezone('Europe/London')
london_dt = loc_dt.astimezone(london_tz)

In [11]:
london_dt

datetime.datetime(2018, 5, 15, 20, 34, tzinfo=<DstTzInfo 'Europe/London' BST+1:00:00 DST>)

In [12]:
f = '%Y-%m-%d %H:%M:%S %Z%z'
datetime.datetime(2018, 5, 12, 12, 15, 0, tzinfo = london_tz).strftime(f)

'2018-05-12 12:15:00 LMT-0001'

#### 시간대를 계산하는 경우

In [13]:
# UTC형식으로 저장 후 출력할 때만 변환

event1 = datetime.datetime(2018, 5, 12, 12, 15, 0, tzinfo = london_tz)
event2 = datetime.datetime(2018, 5, 13, 9, 15, 0, tzinfo = western)
event2 - event1

datetime.timedelta(days=1, seconds=17520)

위의 연산은 잘못된 시간 간격을 계산한다. 각각 시간대가 london과 western으로 다르게 레이블링 되어있기 때문이다.

아래는 localize를 통해 현지화했으므로 유효하다. 위의 연산과 값도 다른 것을 확인할 수 있다.

In [14]:
# localize 후 계산하는 경우
event1 = london_tz.localize( datetime.datetime(2018, 5, 12, 12, 15, 0))
event2 = western.localize(datetime.datetime(2018, 5, 13, 9, 15, 0))
event2 - event1

datetime.timedelta(days=1, seconds=18000)

In [15]:
# localize + UTC 변환
event1 = london_tz.localize((datetime.datetime(2018, 5, 12, 12, 15, 0))).astimezone(datetime.timezone.utc)
event2 = western.localize(datetime.datetime(2018, 5, 13, 9, 15, 0)).astimezone(datetime.timezone.utc)
event2 - event1

datetime.timedelta(days=1, seconds=18000)

**pytz**: pytz는 국가별 시간대와 일반 시간대 목록을 제공

In [ ]:
# 모든 시간대 출력
pytz.common_timezones

In [17]:
# 국가별 시간대 출력
pytz.country_timezones('RU')

['Europe/Kaliningrad',
 'Europe/Moscow',
 'Europe/Kirov',
 'Europe/Astrakhan',
 'Europe/Volgograd',
 'Europe/Saratov',
 'Europe/Ulyanovsk',
 'Europe/Samara',
 'Asia/Yekaterinburg',
 'Asia/Omsk',
 'Asia/Novosibirsk',
 'Asia/Barnaul',
 'Asia/Tomsk',
 'Asia/Novokuznetsk',
 'Asia/Krasnoyarsk',
 'Asia/Irkutsk',
 'Asia/Chita',
 'Asia/Yakutsk',
 'Asia/Khandyga',
 'Asia/Vladivostok',
 'Asia/Ust-Nera',
 'Asia/Magadan',
 'Asia/Sakhalin',
 'Asia/Srednekolymsk',
 'Asia/Kamchatka',
 'Asia/Anadyr']

In [18]:
# 프랑스의 시간대 확인
pytz.country_timezones('fr')

['Europe/Paris']

#### 일광절약을 고려하는 방법

In [19]:
## 시간대
ambig_time = western.localize(datetime.datetime(2002, 10, 27, 1, 30, 00)).astimezone(datetime.timezone.utc)
ambig_time_earlier = ambig_time - datetime.timedelta(hours=1)
ambig_time_later = ambig_time + datetime.timedelta(hours=1)


datetime.datetime(2002, 10, 27, 2, 30, tzinfo=<DstTzInfo 'US/Pacific' PST-1 day, 16:00:00 STD>)

In [20]:
# 초기
ambig_time_earlier.astimezone(western)


datetime.datetime(2002, 10, 27, 1, 30, tzinfo=<DstTzInfo 'US/Pacific' PDT-1 day, 17:00:00 DST>)

In [21]:
# 중기
ambig_time.astimezone(western)


datetime.datetime(2002, 10, 27, 1, 30, tzinfo=<DstTzInfo 'US/Pacific' PST-1 day, 16:00:00 STD>)

In [22]:
# 후기
ambig_time_later.astimezone(western)

datetime.datetime(2002, 10, 27, 2, 30, tzinfo=<DstTzInfo 'US/Pacific' PST-1 day, 16:00:00 STD>)

중기와 후기가 동일하다! 이 경우 is_dst를 통해 일광 절약 여부를 표시해야한다

In [23]:
# is_dst로 일광 절약 여부 확인
ambig_time = western.localize(datetime.datetime(2002, 10, 27, 1, 30, 00), is_dst = True).astimezone(datetime.timezone.utc)
ambig_time_earlier = ambig_time - datetime.timedelta(hours=1)
ambig_time_later = ambig_time + datetime.timedelta(hours=1)


In [24]:

ambig_time_earlier.astimezone(western)


datetime.datetime(2002, 10, 27, 0, 30, tzinfo=<DstTzInfo 'US/Pacific' PDT-1 day, 17:00:00 DST>)

In [25]:
ambig_time.astimezone(western)


datetime.datetime(2002, 10, 27, 1, 30, tzinfo=<DstTzInfo 'US/Pacific' PDT-1 day, 17:00:00 DST>)

In [26]:
ambig_time_later.astimezone(western)

datetime.datetime(2002, 10, 27, 1, 30, tzinfo=<DstTzInfo 'US/Pacific' PST-1 day, 16:00:00 STD>)

## 2.7 사전관찰 방지

사전관찰을 기계적으로 확인할 수는 없다. 끊임 없이 고민하고 경계하는 수 밖에 없다. 평활, 대치, 업 샘플링, 밀고 당김을 할 때마다 해도 되는지, 할 수 있는지, 그 결과가 무엇인지 고민해야한다. 다음은 점검 목록이다.

1. 누락 데이터 대치, 평활할 때 사전 관찰 도입이 결과에 어떤 영향을 끼치는지 고민 및 확인할 것
2. 작은 데이터셋으로 처리의 전체 공정을 구축해보기
3. 각 종류의 데이터셋에 대해 타임스탬프와 관련된 지연 확인. 타임스탬프가 업로드 시점이 아니라 발생 시점에 저장되었는지 등을 파악해야 함.
4. 시간을 인식할 수 있는 에러(롤링) 검사 또는 교차 검증 사용 ->11장
5. 의도적으로 사전 관찰 도입하여 모델 관찰. -> 감 익히기
6. 피처를 추가하면서 성능 향상 관찰. 사전 관찰일 경우 특정 피처가 특별한 이유 없이 매우 좋을 수 있음.